In [1]:
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from tqdm import tqdm, trange
import torch
import torch.nn.functional as F
#from retrieval_model import BertEncoder
from transformers import AdamW, TrainingArguments, get_linear_schedule_with_warmup
from torch import nn
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm import tqdm, trange
import pickle
import os, json

In [12]:
# qg_dataset = pd.read_csv('qg_high_prob_answers.csv')
# qg = qg_dataset.reset_index()
qg

,index,Unnamed: 0,context,question,answer,sts_score,mrc_answers
0,0,1,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리구는 경상남도 울주군 어느면에서 출토되었다고 전해지는가?,웅촌면,0.418,"('웅촌면', (45, 48))"
1,1,3,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리구를 만든 승표 부부는 어떤 신분이었어?,시주자,0.187,"('시주자', (146, 149))"
2,2,5,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리구는 세로로 몇 줄 쓰여있어?,세 줄,0.618,"('세 줄', (132, 134))"
3,3,6,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리구를 소장하고 있는 곳은?,동아대학교박물관,0.501,"('동아대학교박물관', (13, 17))"
4,4,9,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리구에는 무엇으로 쓴 명문이 세로로 세 줄에 걸쳐 쓰여있어?,청화안료,0.251,"('청화안료', (115, 119))"
...,...,...,...,...,...,...,...
23783,23783,112799,1947년 6월 미군정 사령관 존 하지는 이승만에게 이승만과 김구가 계획 중이라는 ...,여운형이 최우선으로 생각했던 것은?,국민,0.352,"('국민', (162, 163))"
23784,23784,113160,전날 밤부터 난지도쪽으로 한강을 도하한 1개 연대 규모의 북한군이 오류동 정면으로 ...,북한군이 공격에 앞서 비행장쪽에서 그들의 보병에 화력을 지원하던 전차 2대를 야음을...,남쪽,0.340,"('남쪽', (83, 84))"
23785,23785,113778,1969년에 연정에 합의한 사민당과 자유민주당의 사회자유 연정은 기존 외교 정책에서...,1969년 사민당과 자유민주당의 사회자유 연정이 우호적 입장을 취한 나라는 동독과 ...,소련,0.287,"('소련', (64, 65))"
23786,23786,114062,오이 샌드위치는 영국이 기원인 샌드위치의 하나이다. 현대의 다른 종류의 오이샌드위치...,현대의 오이샌드위치는 대부분 어느 나라에서 만들어지나요?,미국,0.354,"('미국', (37, 38))"


In [19]:
len(qg['question'].unique())

23782

In [17]:
origin_train_dataset = load_from_disk('data/train_dataset')['train']
origin_train_dataset

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 3952
})

In [24]:
new_t = load_from_disk('data/new_train_dataset')['train']

In [27]:
context_mapping_qg_to_origin = {}
for i in tqdm(range(len(qg))):
    for j in range(len(origin_train_dataset)):
        if qg['context'][i] == new_t[j]['context']:
            context_mapping_qg_to_origin[i] = j
len(context_mapping_qg_to_origin)

100%|██████████| 23788/23788 [3:04:58<00:00,  2.14it/s]  


23787

In [55]:
len(context_mapping_qg_to_origin)

23788

In [53]:
# for i in range(len(new_t)):
#     if '호박 잎' in new_t[i]['context']:
#         print(i)
# new_t[1430]['context']
context_mapping_qg_to_origin[23566] = 1430

In [44]:
# print(origin_train_dataset[context_mapping_qg_to_origin[23566]]['context'])
qg['context'][23566]

'호박 잎은 심장모양 또는 콩팥모양으로, 커다랗고 거친 털로 덮여 있으며 가장자리가 5개로 얕게 갈라지고 갈래조각에 톱니가 있다 잎자루가 길고, 어긋나기한다. 꽃은 암수한그루로서, 수꽃과 암꽃이 따로 핀다 6월부터 서리가 내릴 때까지 종 모양의 노란 꽃이 잎겨드랑이에 1개씩 달려 핀다 수꽃에만 있는 화분을 벌이 암꽃에 옮기면 수분이 되고, 수분된 암꽃에서 호박이 자란다 암꽃 하나하나가 단 하루만 피어 수분할 수 있는 데다가 호박꽃 대부분이 수꽃이기 때문에 실제로 호박을 생성하는 꽃은 몇 송이밖에 없다 수꽃은 꽃대가 길며 꽃받침통이 얕고 갈래조각의 기부가 꽃부리에 붙어 있으며 암꽃은 꽃대가 짧고 밑부분에 긴 씨방이 있으며 꽃받침갈래조각이 다소 잎같이 된다. 덩굴은 단면이 오각형이고 긴 흰색 솜털이 나있으며 덩굴손으로 감으면서 자라지만 개량된 것은 덩굴성이 아닌 것도 있다. 호박 열매는 여러 종류가 있으며, 모양과 빛깔도 품종에 따라 다르다 원형 또는 타원형으로 껍질이 단단하거나 무르다 속은 결이 거칠고 끈끈한 섬유질로 이루어져 있으며 열매 가운데에 씨가 들어 있다 열매의 무게는 대개 7-14kg 정도이지만 90kg이 넘게 나가는 것도 있다 열매는 대개 오렌지색을 띠며, 노란색·녹색·흰색 등 다른 색깔을 띠는 것도 많다 익으면 바깥면이 주로 짙은 황갈색을 띤다 편평하고 맛이 좋은 씨가 많이 들어 있다 완숙된 종자로 번식한다'

In [54]:
# print(origin_train_dataset[context_mapping_qg_to_origin[1]]['context'])
# qg['context'][1]

for i in range(len(qg)):
    if qg['context'][i] != new_t[context_mapping_qg_to_origin[i]]['context']:
        print(i)

23566


In [57]:
cmqto = []
for i in range(len(qg)):
    cmqto.append(context_mapping_qg_to_origin[i])

In [59]:
qg['context_mapping_qg_to_origin'] = cmqto
qg.to_csv('preprob_qg.csv')

In [26]:
context_mapping_qg_to_origin

{0: 3237, 1: 3237, 2: 3237, 3: 3237, 4: 3237, 5: 3237, 6: 3237, 7: 4}

In [15]:
qg_query = list(qg['question'])
len(qg_query)

23788

In [ ]:
'answers': {'text': ['하원'], 'answer_start': [229]}}

In [19]:
qg1 = load_from_disk('qg_dataset_top_4')
qg2 = pd.DataFrame(data={'answers': qg1['answers'], 'context': qg1['context'], 'id': qg1['id'], 'question': qg1['question']})
qg2

,answers,context,id,question
0,"{'answer_start': [235], 'text': ['하원']}",미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,mrc-1-000067,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
1,"{'answer_start': [212], 'text': ['《경영의 실제》']}",'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,mrc-0-004397,현대적 인사조직관리의 시발점이 된 책은?
2,"{'answer_start': [510], 'text': ['백성']}",강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,mrc-1-000362,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?
3,"{'answer_start': [625], 'text': ['중국']}","불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",mrc-0-001510,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?
4,"{'answer_start': [30], 'text': ['4개']}",동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,mrc-0-000823,명문이 적힌 유물을 구성하는 그릇의 총 개수는?
...,...,...,...,...
27720,"{'answer_start': [162], 'text': ['국민']}",1947년 6월 미군정 사령관 존 하지는 이승만에게 이승만과 김구가 계획 중이라는 ...,mrc-qg-23783,여운형이 최우선으로 생각했던 것은?
27721,"{'answer_start': [83], 'text': ['남쪽']}",전날 밤부터 난지도쪽으로 한강을 도하한 1개 연대 규모의 북한군이 오류동 정면으로 ...,mrc-qg-23784,북한군이 공격에 앞서 비행장쪽에서 그들의 보병에 화력을 지원하던 전차 2대를 야음을...
27722,"{'answer_start': [64], 'text': ['소련']}",1969년에 연정에 합의한 사민당과 자유민주당의 사회자유 연정은 기존 외교 정책에서...,mrc-qg-23785,1969년 사민당과 자유민주당의 사회자유 연정이 우호적 입장을 취한 나라는 동독과 ...
27723,"{'answer_start': [37], 'text': ['미국']}",오이 샌드위치는 영국이 기원인 샌드위치의 하나이다. 현대의 다른 종류의 오이샌드위치...,mrc-qg-23786,현대의 오이샌드위치는 대부분 어느 나라에서 만들어지나요?


In [2]:
qg = pd.read_csv('preprob_qg.csv')
qg

,Unnamed: 0,index,Unnamed: 0.1,context,question,answer,sts_score,mrc_answers,context_mapping_qg_to_origin
0,0,0,1,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리구는 경상남도 울주군 어느면에서 출토되었다고 전해지는가?,웅촌면,0.418,"('웅촌면', (45, 48))",3237
1,1,1,3,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리구를 만든 승표 부부는 어떤 신분이었어?,시주자,0.187,"('시주자', (146, 149))",3237
2,2,2,5,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리구는 세로로 몇 줄 쓰여있어?,세 줄,0.618,"('세 줄', (132, 134))",3237
3,3,3,6,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리구를 소장하고 있는 곳은?,동아대학교박물관,0.501,"('동아대학교박물관', (13, 17))",3237
4,4,4,9,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리구에는 무엇으로 쓴 명문이 세로로 세 줄에 걸쳐 쓰여있어?,청화안료,0.251,"('청화안료', (115, 119))",3237
...,...,...,...,...,...,...,...,...,...
23783,23783,23783,112799,1947년 6월 미군정 사령관 존 하지는 이승만에게 이승만과 김구가 계획 중이라는 ...,여운형이 최우선으로 생각했던 것은?,국민,0.352,"('국민', (162, 163))",263
23784,23784,23784,113160,전날 밤부터 난지도쪽으로 한강을 도하한 1개 연대 규모의 북한군이 오류동 정면으로 ...,북한군이 공격에 앞서 비행장쪽에서 그들의 보병에 화력을 지원하던 전차 2대를 야음을...,남쪽,0.340,"('남쪽', (83, 84))",1992
23785,23785,23785,113778,1969년에 연정에 합의한 사민당과 자유민주당의 사회자유 연정은 기존 외교 정책에서...,1969년 사민당과 자유민주당의 사회자유 연정이 우호적 입장을 취한 나라는 동독과 ...,소련,0.287,"('소련', (64, 65))",3613
23786,23786,23786,114062,오이 샌드위치는 영국이 기원인 샌드위치의 하나이다. 현대의 다른 종류의 오이샌드위치...,현대의 오이샌드위치는 대부분 어느 나라에서 만들어지나요?,미국,0.354,"('미국', (37, 38))",1168


In [ ]:
new_dict = {}

for i in range(len(qg)):
    qg['context'][i]

In [6]:
len(qg['context'].unique())

1625

In [11]:
from collections import defaultdict

score_dict = dict()
id_dict = dict()

for i in range(len(qg)):
    row = qg.iloc[i]
    context = row['context']
    sts_score = row['sts_score']
    if score_dict.get(context, 0)<sts_score:
        score_dict[context] = sts_score
        id_dict[context] = i

In [21]:
qg2 = qg.loc[id_dict.values()]
qg2 = qg2.reset_index()
qg2

,level_0,Unnamed: 0,index,Unnamed: 0.1,context,question,answer,sts_score,mrc_answers,context_mapping_qg_to_origin
0,2,2,2,5,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리구는 세로로 몇 줄 쓰여있어?,세 줄,0.618,"('세 줄', (132, 134))",3237
1,14,14,14,34,"자신의 후견자 겸 제자인 루돌프 대공을 위해, 1809년에 프랑스 군대의 포격이 쏟...",교향곡 다장조를 작곡한 사람이 누구야?,베토벤,0.489,"('베토벤', (83, 84))",3122
2,34,34,34,73,고조선의 도읍지는 여러 차례 이동한 것으로 기록되어 있다. 《삼국유사》는 《고기》를...,단군왕검이 1천 5백년간 나라를 다스렸다고 기록된 책은?,삼국유사,0.631,"('삼국유사', (33, 35))",3438
3,53,53,53,110,너비아니(nubianie)는 궁중식 불고기로 알려진 한국 요리이다. 쇠고기를 칼로 ...,쇠고기의 두께를 알려줘?,5 mm,0.470,"('5 mm', (200, 202))",1729
4,67,67,67,151,삼손은 단 지파의 히브리인으로 그의 어머니 하술렐보니(Hazelelponit)에 의...,나실인이 되기로 서약된 사람은?,삼손,0.404,"('삼손', (13, 15))",983
...,...,...,...,...,...,...,...,...,...,...
1620,23783,23783,23783,112799,1947년 6월 미군정 사령관 존 하지는 이승만에게 이승만과 김구가 계획 중이라는 ...,여운형이 최우선으로 생각했던 것은?,국민,0.352,"('국민', (162, 163))",263
1621,23784,23784,23784,113160,전날 밤부터 난지도쪽으로 한강을 도하한 1개 연대 규모의 북한군이 오류동 정면으로 ...,북한군이 공격에 앞서 비행장쪽에서 그들의 보병에 화력을 지원하던 전차 2대를 야음을...,남쪽,0.340,"('남쪽', (83, 84))",1992
1622,23785,23785,23785,113778,1969년에 연정에 합의한 사민당과 자유민주당의 사회자유 연정은 기존 외교 정책에서...,1969년 사민당과 자유민주당의 사회자유 연정이 우호적 입장을 취한 나라는 동독과 ...,소련,0.287,"('소련', (64, 65))",3613
1623,23786,23786,23786,114062,오이 샌드위치는 영국이 기원인 샌드위치의 하나이다. 현대의 다른 종류의 오이샌드위치...,현대의 오이샌드위치는 대부분 어느 나라에서 만들어지나요?,미국,0.354,"('미국', (37, 38))",1168


In [33]:
qg2.to_csv('qg_filter.csv')

In [22]:
qg_datas = []
for i in range(len(qg2)):
    row = qg2.iloc[i]
    qg_datas.append({
            'id': 'mrc-qg-' + str(row['index']),
            'context': row['context'],
            'question': row['question'],
            'answers': {
                'text': [row['answer']],
                'answer_start': [int(eval(row['mrc_answers'])[1][0])]
            }
        })
tmp_pd =pd.DataFrame(qg_datas)
tmp_pd

,id,context,question,answers
0,mrc-qg-2,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,계사명 사리구는 세로로 몇 줄 쓰여있어?,"{'text': ['세 줄'], 'answer_start': [132]}"
1,mrc-qg-14,"자신의 후견자 겸 제자인 루돌프 대공을 위해, 1809년에 프랑스 군대의 포격이 쏟...",교향곡 다장조를 작곡한 사람이 누구야?,"{'text': ['베토벤'], 'answer_start': [83]}"
2,mrc-qg-34,고조선의 도읍지는 여러 차례 이동한 것으로 기록되어 있다. 《삼국유사》는 《고기》를...,단군왕검이 1천 5백년간 나라를 다스렸다고 기록된 책은?,"{'text': ['삼국유사'], 'answer_start': [33]}"
3,mrc-qg-53,너비아니(nubianie)는 궁중식 불고기로 알려진 한국 요리이다. 쇠고기를 칼로 ...,쇠고기의 두께를 알려줘?,"{'text': ['5 mm'], 'answer_start': [200]}"
4,mrc-qg-67,삼손은 단 지파의 히브리인으로 그의 어머니 하술렐보니(Hazelelponit)에 의...,나실인이 되기로 서약된 사람은?,"{'text': ['삼손'], 'answer_start': [13]}"
...,...,...,...,...
1620,mrc-qg-23783,1947년 6월 미군정 사령관 존 하지는 이승만에게 이승만과 김구가 계획 중이라는 ...,여운형이 최우선으로 생각했던 것은?,"{'text': ['국민'], 'answer_start': [162]}"
1621,mrc-qg-23784,전날 밤부터 난지도쪽으로 한강을 도하한 1개 연대 규모의 북한군이 오류동 정면으로 ...,북한군이 공격에 앞서 비행장쪽에서 그들의 보병에 화력을 지원하던 전차 2대를 야음을...,"{'text': ['남쪽'], 'answer_start': [83]}"
1622,mrc-qg-23785,1969년에 연정에 합의한 사민당과 자유민주당의 사회자유 연정은 기존 외교 정책에서...,1969년 사민당과 자유민주당의 사회자유 연정이 우호적 입장을 취한 나라는 동독과 ...,"{'text': ['소련'], 'answer_start': [64]}"
1623,mrc-qg-23786,오이 샌드위치는 영국이 기원인 샌드위치의 하나이다. 현대의 다른 종류의 오이샌드위치...,현대의 오이샌드위치는 대부분 어느 나라에서 만들어지나요?,"{'text': ['미국'], 'answer_start': [37]}"


In [32]:
tmp_pd.to_csv('qg_filter.csv')

In [17]:
h1h = load_from_disk('train_with_origin_gt_add_top_k_passage/not_include_answer_passage_train_es_top_4')
iis = []
ccs = []
qqs = []
aas = []

iis = h1h['id']
ccs = h1h['context']
qqs = h1h['question']
aas = h1h['answers']
qgqg3 = pd.DataFrame(data={'answers': aas, 'context': ccs, 'id': iis, 'question': qqs})
qgqg3

,answers,context,id,question
0,"{'answer_start': [235], 'text': ['하원']}",미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,mrc-1-000067,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
1,"{'answer_start': [212], 'text': ['《경영의 실제》']}",'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,mrc-0-004397,현대적 인사조직관리의 시발점이 된 책은?
2,"{'answer_start': [510], 'text': ['백성']}",강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,mrc-1-000362,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?
3,"{'answer_start': [625], 'text': ['중국']}","불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",mrc-0-001510,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?
4,"{'answer_start': [30], 'text': ['4개']}",동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,mrc-0-000823,명문이 적힌 유물을 구성하는 그릇의 총 개수는?
...,...,...,...,...
3947,"{'answer_start': [317], 'text': ['지질 구조']}",이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,mrc-0-005285,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?
3948,"{'answer_start': [616], 'text': ['뉴저지']}",애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,mrc-1-000699,애니는 어디서 태어났는가?
3949,"{'answer_start': [255], 'text': ['칼 에케르트']}",1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,mrc-0-003429,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?
3950,"{'answer_start': [476], 'text': ['슬라브 신화']}",원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,mrc-0-003956,드래곤의 암수 구분이 있는 신화는?


In [23]:
ii = list(tmp_pd['id'])
cc = list(tmp_pd['context'])
qqu = list(tmp_pd['question'])
aan = list(tmp_pd['answers'])

aas.extend(aan)
ccs.extend(cc)
iis.extend(ii)
qqs.extend(qqu)

qgqg4 = pd.DataFrame(data={'answers': aas, 'context': ccs, 'id': iis, 'question': qqs})
qgqg4

,answers,context,id,question
0,"{'answer_start': [235], 'text': ['하원']}",미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,mrc-1-000067,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
1,"{'answer_start': [212], 'text': ['《경영의 실제》']}",'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,mrc-0-004397,현대적 인사조직관리의 시발점이 된 책은?
2,"{'answer_start': [510], 'text': ['백성']}",강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,mrc-1-000362,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?
3,"{'answer_start': [625], 'text': ['중국']}","불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",mrc-0-001510,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?
4,"{'answer_start': [30], 'text': ['4개']}",동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,mrc-0-000823,명문이 적힌 유물을 구성하는 그릇의 총 개수는?
...,...,...,...,...
5572,"{'text': ['국민'], 'answer_start': [162]}",1947년 6월 미군정 사령관 존 하지는 이승만에게 이승만과 김구가 계획 중이라는 ...,mrc-qg-23783,여운형이 최우선으로 생각했던 것은?
5573,"{'text': ['남쪽'], 'answer_start': [83]}",전날 밤부터 난지도쪽으로 한강을 도하한 1개 연대 규모의 북한군이 오류동 정면으로 ...,mrc-qg-23784,북한군이 공격에 앞서 비행장쪽에서 그들의 보병에 화력을 지원하던 전차 2대를 야음을...
5574,"{'text': ['소련'], 'answer_start': [64]}",1969년에 연정에 합의한 사민당과 자유민주당의 사회자유 연정은 기존 외교 정책에서...,mrc-qg-23785,1969년 사민당과 자유민주당의 사회자유 연정이 우호적 입장을 취한 나라는 동독과 ...
5575,"{'text': ['미국'], 'answer_start': [37]}",오이 샌드위치는 영국이 기원인 샌드위치의 하나이다. 현대의 다른 종류의 오이샌드위치...,mrc-qg-23786,현대의 오이샌드위치는 대부분 어느 나라에서 만들어지나요?


In [24]:
Dataset.from_pandas(qgqg4).save_to_disk(f"qg_filterings")
!tar -cvf qg_filterings.tar qg_filterings

qg_filterings/
qg_filterings/state.json
qg_filterings/dataset_info.json
qg_filterings/dataset.arrow


In [25]:
aa2 = load_from_disk('qg_filterings')
aa2

Dataset({
    features: ['answers', 'context', 'id', 'question'],
    num_rows: 5577
})

In [30]:
aa2[5575]

{'context': '오이 샌드위치는 영국이 기원인 샌드위치의 하나이다. 현대의 다른 종류의 오이샌드위치는 대부분 미국에서 만들어지며, 크림치즈, 다진 딜이나 향신료, 호밀빵, 연어를 넣어 만들거나 빵 껍질을 제거하지 않기도 한다. 오이, 크림치즈를 섞은 녹색의 부드러운 스프레드인 ‘베네딕틴’을 넣고 만들기도 한다. 전통적인 오이 샌드위치는 버터를 얇게 바른 두 장의 흰 식빵 사이에 종잇장처럼 얇게 썬 오이를 넣고 만든다. 얇은 식빵을 사용하는 게 중요하므로 날이 넓은 칼로 밀도 높은 흰 풀먼 식빵을 잘라 쓴다. 오이를 자르기 전에 껍질을 벗겨 내거나 길게 포크 자국을 낸다. 오이 즙 때문에 축축해지지 않게 식빵에 버터를 골고루 얇게 바르고, 소금과 레몬 즙을 살짝 뿌려둔 얇게 썬 오이는 먹기 직전 식빵에 넣어야 물기 없이 깔끔하다. 식빵 껍질은 깨끗이 잘라내고, 대각선으로 두 번 잘라 삼각형의 샌드위치 네 조각을 만든다. 오이 샌드위치는 보통 간식으로 먹거나 오후에 차를 마실 때 같이 먹는데, 이는 오후 4시나 저녁 식사를 하기 전인 초저녁쯤이다. 또한, 오이 샌드위치는 잉글랜드에서 크리켓 경기의 휴식 시간에 흔히 먹는다. 인도에서도 영국의 영향으로, 크리켓 경기나 주말 소풍 때 오이 샌드위치를 많이 먹는다. 인도식 오이 샌드위치에는 녹색 처트니를 넣고 때로는 얇게 썬 삶은 감자를 넣기도 한다. 오이의 시원한 특성 때문에, 오이 샌드위치는 인도의 어느 지역에서는 따뜻한 날씨나 여름철에 자주 먹는다. 인도 항공에서는 단거리 국내 비행 시 오이 샌드위치를 채식주의자용 기내식으로 내기도 했다.',
 'answers': {'answer_start': [37], 'text': ['미국']},
 'question': '현대의 오이샌드위치는 대부분 어느 나라에서 만들어지나요?',
 'id': 'mrc-qg-23786'}